Logistic Regression - Employeement leave

First we import all module that we need 

In [36]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

then import dataset resource

In [37]:
df = pd.read_csv("../CSV file/HumanResource.csv")
df.head()

df = df.dropna()

df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [38]:
df.isnull().sum()
df.dtypes
df.groupby(df.left)["satisfaction_level"].describe()

,count,mean,std,min,25%,50%,75%,max
left,,,,,,,,
0,11428.0,0.666810,0.217104,0.12,0.54,0.69,0.84,1.00
1,3571.0,0.440098,0.263933,0.09,0.13,0.41,0.73,0.92


i notice that satisfaction level can be range value like < 50 isnt satisfaction and > 50 is satisfaction if we look at mean value from left and stay worker, so ill convert it

In [39]:
bins= [0, 0.5, 1]
labels = ["not satisfaction", "satisfaction"]
df['satisfaction_level'] = pd.cut(df['satisfaction_level'], bins=bins, labels=labels, right=False)

df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,not satisfaction,0.53,2,157,3,0,1,0,sales,low
1,satisfaction,0.86,5,262,6,0,1,0,sales,medium
2,not satisfaction,0.88,7,272,4,0,1,0,sales,medium
3,satisfaction,0.87,5,223,5,0,1,0,sales,low
4,not satisfaction,0.52,2,159,3,0,1,0,sales,low


i'll comparing each column mean from left workers and stay workers for knowing each columns impact on left workers

In [40]:
df.groupby(df.left).mean()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
left,,,,,,
0,0.715473,3.786664,199.060203,3.380032,0.175009,0.026251
1,0.718113,3.855503,207.419210,3.876505,0.047326,0.005321


then eliminate all columns that useless in this case

In [41]:
df = df[["satisfaction_level", "average_montly_hours", "time_spend_company", "promotion_last_5years", "salary", "left"]]
df.head()

,satisfaction_level,average_montly_hours,time_spend_company,promotion_last_5years,salary,left
0,not satisfaction,157,3,0,low,1
1,satisfaction,262,6,0,medium,1
2,not satisfaction,272,4,0,medium,1
3,satisfaction,223,5,0,low,1
4,not satisfaction,159,3,0,low,1


now i do one hot encoder for satisfaction columns that i already convert and salary columns which contain string

In [42]:
#OHE Salary
dummies = pd.get_dummies(df.salary)
df = pd.concat([dummies.drop(["low"], axis="columns"), df], axis="columns")
df.head()

#OHE satisfaction
dummies = pd.get_dummies(df.satisfaction_level)
df = pd.concat([dummies.drop(["not satisfaction"], axis="columns"), df], axis="columns")
df.head()

df.drop(["satisfaction_level", "salary"], axis="columns", inplace=True)

now i do scalling for data 

In [43]:
#sc = StandardScaler()
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
df[["average_montly_hours", "time_spend_company"]] = sc.fit_transform(df[["average_montly_hours", "time_spend_company"]])

df0 = df[df["left"] == 0]
df1 = df[df["left"]== 1]

df1.shape

(3571, 7)

i'll notice there is imbalance between left workers and stay workers (11428/3571), so i'll handle it using oversampling minority method for this imbalance dataset case

In [44]:
#oversampling
countcls =  df.left.value_counts()
df0 = df[df["left"] == 0]
df1 = df[df["left"]== 1]

df0.shape
df1.shape

df1over = df1.sample(countcls[0], replace=True)
oversml = pd.concat([df0, df1over], axis=0)
oversml.shape

oversml = oversml.dropna()
oversml

,satisfaction,high,medium,average_montly_hours,time_spend_company,promotion_last_5years,left
2000,1,0,0,0.279320,-0.341235,0,0
2001,1,0,0,0.019016,-0.341235,0,0
2002,0,0,0,-0.161196,-0.341235,0,0
2003,1,0,0,0.920071,-0.341235,0,0
2004,0,0,0,0.259297,-1.026126,0,0
...,...,...,...,...,...,...,...
1615,0,0,0,2.121478,0.343655,0,1
243,0,0,0,-1.462720,-0.341235,0,1
1255,0,0,0,-1.182392,-0.341235,0,1
14819,0,0,1,-1.342579,-0.341235,0,1


Features and target split 

In [45]:
x_balance = oversml.drop(["left", "average_montly_hours"], axis="columns")
y_balance = oversml["left"]

x_balance.shape

(22856, 5)

search best parameters for logistic regression with Gridsearch

In [46]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2', "l1"]
c_values = [70, 100, 60, 50, 20, 30, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x_balance, y_balance)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.760850 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.756869 (0.011283) with: {'C': 70, 'penalty': 'l2', 'solver': 'newton-cg'}
0.756869 (0.011283) with: {'C': 70, 'penalty': 'l2', 'solver': 'lbfgs'}
0.756869 (0.011283) with: {'C': 70, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 70, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 70, 'penalty': 'l1', 'solver': 'lbfgs'}
0.756869 (0.011283) with: {'C': 70, 'penalty': 'l1', 'solver': 'liblinear'}
0.756869 (0.011283) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.756869 (0.011283) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.756869 (0.011283) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
0.000000 (0.000000) with: {'C': 100, 'penalty': 'l1', 'solver': 'lbfgs'}
0.756869 (0.011283) with: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
0.756869 (0.011283) with:

Train Test split and model fitting

In [47]:
xtr, xte, ytr, yte = train_test_split(x_balance, y_balance, test_size=.25, random_state=10)

Lr = LogisticRegression(C=.05, penalty="l2", solver="newton-cg")
Lr.fit(xtr, ytr)
Lr.score(xte, yte)

0.7528876443822191

model report

In [48]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(yte, Lr.predict(xte)))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76      2865
           1       0.76      0.73      0.75      2849

    accuracy                           0.75      5714
   macro avg       0.75      0.75      0.75      5714
weighted avg       0.75      0.75      0.75      5714

